In [ ]:
# FairLoans – 01_explore.ipynb

Initial data exploration, profiling, and bias-related inspection.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style='whitegrid')


In [2]:
df = pd.read_csv('../data/loan_dataset.csv')  # Update path if needed
df.head()

,ID,Gender,Race,Age,Age_Group,Income,Credit_Score,Loan_Amount,Employment_Type,Education_Level,Citizenship_Status,Language_Proficiency,Disability_Status,Criminal_Record,Zip_Code_Group,Loan_Approved
0,1,Female,White,74,Over 60,112117,641,378691,Full-time,Graduate,Citizen,Fluent,No,No,High-income Suburban,Approved
1,2,Female,White,36,25-60,155943,664,441856,Self-employed,High School,Permanent Resident,Limited,No,No,Urban Professional,Denied
2,3,Female,White,27,25-60,130933,550,401760,Full-time,Graduate,Citizen,Fluent,No,No,Urban Professional,Denied
3,4,Male,White,67,Over 60,165816,804,159264,Gig,Some College,Citizen,Fluent,No,No,Urban Professional,Approved
4,5,Male,Multiracial,57,25-60,81510,694,256742,Full-time,Some College,Visa Holder,Fluent,No,No,Rural,Denied


In [3]:
# Dataset info
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    10000 non-null  int64 
 1   Gender                10000 non-null  object
 2   Race                  10000 non-null  object
 3   Age                   10000 non-null  int64 
 4   Age_Group             10000 non-null  object
 5   Income                10000 non-null  int64 
 6   Credit_Score          10000 non-null  int64 
 7   Loan_Amount           10000 non-null  int64 
 8   Employment_Type       10000 non-null  object
 9   Education_Level       10000 non-null  object
 10  Citizenship_Status    10000 non-null  object
 11  Language_Proficiency  10000 non-null  object
 12  Disability_Status     10000 non-null  object
 13  Criminal_Record       10000 non-null  object
 14  Zip_Code_Group        10000 non-null  object
 15  Loan_Approved         10000 non-null 

,ID,Gender,Race,Age,Age_Group,Income,Credit_Score,Loan_Amount,Employment_Type,Education_Level,Citizenship_Status,Language_Proficiency,Disability_Status,Criminal_Record,Zip_Code_Group,Loan_Approved
count,10000.00000,10000,10000,10000.000000,10000,10000.000000,10000.000000,10000.000000,10000,10000,10000,10000,10000,10000,10000,10000
unique,NaN,3,6,NaN,3,NaN,NaN,NaN,5,4,3,2,2,2,5,2
top,NaN,Female,White,NaN,25-60,NaN,NaN,NaN,Full-time,Bachelor's,Citizen,Fluent,No,No,High-income Suburban,Denied
freq,NaN,4910,6008,NaN,5558,NaN,NaN,NaN,6535,3448,8552,8508,8804,9211,2560,5685
mean,5000.50000,NaN,NaN,49.971200,NaN,100637.253200,673.662200,252688.666000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2886.89568,NaN,NaN,18.746901,NaN,43560.318163,101.437588,115148.438875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.00000,NaN,NaN,18.000000,NaN,25012.000000,500.000000,50140.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2500.75000,NaN,NaN,34.000000,NaN,62851.000000,585.000000,152507.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5000.50000,NaN,NaN,50.000000,NaN,100501.500000,672.000000,252844.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,7500.25000,NaN,NaN,66.000000,NaN,138240.500000,762.000000,354880.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print('Missing Values:')
print(df.isnull().sum())

print('\nDuplicate Rows:', df.duplicated().sum())


Missing Values:
ID                      0
Gender                  0
Race                    0
Age                     0
Age_Group               0
Income                  0
Credit_Score            0
Loan_Amount             0
Employment_Type         0
Education_Level         0
Citizenship_Status      0
Language_Proficiency    0
Disability_Status       0
Criminal_Record         0
Zip_Code_Group          0
Loan_Approved           0
dtype: int64

Duplicate Rows: 0


In [5]:
## 🔍 Sensitive Feature Analysis

In [6]:
sensitive_columns = ['gender', 'race', 'region', 'loan_status']

for col in sensitive_columns:
    if col in df.columns:
        print(f"\n{col} Distribution:")
        print(df[col].value_counts(normalize=True) * 100)
        sns.countplot(data=df, x=col)
        plt.title(f"{col} Distribution")
        plt.show()
